In [5]:
# packages
import numpy as np
import pandas as pd
import os

# database
import yfinance as yf
from sqlalchemy import create_engine, inspect

# visualisation
import seaborn as sns
import plotly.express as px
import matplotlib
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
plt.style.use('fivethirtyeight')
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

from tabulate import tabulate
import quadprog
from scipy.optimize import minimize

/Users/jackstevenson/CQF/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
# create the database

# # download (only need close)
df = yf.download('IYE IYM IYJ IYC IYK IYH IYF IYW IYZ IDU IYR ^IRX ^VIX ^BCOM AGG', start='2010-09-24', end='2024-06-30')['Close']

# What are the 11 GICS sectors
# Energy - iShares U.S. Energy ETF (IYE)
# Materials - iShares U.S. Materials ETF (IYM)
# Industrials - iShares U.S. Industrials ETF (IYJ)
# Consumer Discretionary - iShares U.S. Consumer Disc ETF (IYC)
# Consumer Staples - iShares U.S. Consumer Staples ETF (IYK)
# Health Care - iShares U.S. Healthcare ETF (IYH)
# Financials - iShares U.S. Financials ETF (IYF)
# Information Technology - iShares U.S. Technology ETF (IYW)
# Communication Services - iShares U.S. Telecom ETF (IYZ)
# Utilities - iShares U.S. Utilities ETF (IDU)
# Real Estate - iShares US Real Estate ETF (IYR)

# Exogenous
# Volatility - CBOE Volatility Index - (^VIX)
# Commodity - Bloomberg Commodity Index Total Return (^BCOM)
# Bonds - iShares Core US Aggregate Bond ETF (AGG)


# # reset index
df.reset_index(inplace=True)

# # create our engine
engine = create_engine("sqlite:///project_portfolio.db")

# # if_exits will overwrite the pre-existing table
df.to_sql('portfolio_data', engine, if_exists='replace', index=False)

# # we use this to inspect our data beforehand
# inspector = inspect(engine)

# table_names = inspector.get_table_names()

# print(table_names)

[*********************100%%**********************]  15 of 15 completed


3463

#### Quotes

BLack Litterman (1990)

'Our model does not assume thatthe world is always at CAPM equilibrium, but rather that when ex-pected returns move away from their equilibrium values, imbal-ances in marketswill tend to pushthem back'

'The investor's benchmark defines the point of origin for measuring this risk. In other words, it represents the minimum-risk portfolio.'

The appropriate measure of risk is the tracking error. 

#### Idzorek

'In the absence of constraints, the Black-Litterman model only recommends a departure from an asset’s market capitalization weight if it is the subject of a view. ' 

1.1 Portfolio Choice

We will attempt to replicate the S&P 500 by creating a portfolio composed of the individual GICs sectors. We will implant an additional three exogenous factors in a Volatility Index, Comoodity Index and a Bond Index. 

We will use CBOE Volatility Index (VIX) to model volatility, effectively functioning like a hedge option that should only return when other factor are down. We will use the Bloomberg Commodity Index Total Return (BCOM) as a proxy to introduce commoditiy shocks into the portfolio choices. Finally we will use a generic ETF iShares Core US Aggregate Bond (AGG) to represent the Bond markets. 

In addition to these three factors we will also introduce four derived Fama-French factors. The daily return for each of these assets has been calculated based on pre-defined portfolios of assets. The four factors are Small Minus Big (SMB), High Minus Low (HML), Robust Minus Weak (RMW), and Conservative Minus Aggressive (CMA). 

HML is the average return on the two values portfolios (Small Value & Big Value) minus the average return on the two growth portfolios (Small Growth & Big Growth).  RMW (Robust Minus Weak) is the average return on the two robust operating profitability portfolios (Small Robus & Big Robust) minus the average return on the two weak operating profitability portfolios (Small Weak & Big Weak). CMA (Conservative Minus Aggressive) is the average return on the two conservative investment portfolios (Small Conservative & Big Conservative) minus the average return on the two aggressive investment portfolios (Small Aggresive & Big Aggressive). SMB is the average return of the six previously mentioned small portfolios plus three additional neutral variants for a total of nine small stock portfolios minus the average return on a comparative portfolio of nine big stock portfolios with identical categories. 

Together this we will give us a total of seven assets externals to our 11 GICs Sectors which should replicate the S&P 500 portfolio. We will dedicate 93% of the portfolio to these 11 GICs sectors (weighted by their constitute parts of the benchmark index) and we will equally split the remaining 7% to an equal allocation across our three exogenous factors and 'investable' Fama-French factors. 


Fig 1. Table of Assets

In [7]:
# TODO
# 
# Generate Correlation between our 18 factors
# Present P&L Returns & Back Testing vs Market (S&P 500)
# Performance , Plots of Rollings Beta and changing Alpha
# Discuss CAPM and the Fama French model 

# Black Litterman 
# Compare the performance of the portfolio vs factors and market (rolling Beta), indepdenent and jointly (Post 2022)


<!--  -->

#### 2.1. What is the Black-Litterman Portfolio

The initial formulation of the Black-Litterman model for Portfolio construction was described in the 1992 paper. The paper described an intuitive solution by combining the mean-variance opitimisation framework of Markowitz and the capital asset pricing model (CAPM) of Sharpe and Lintner. The model uses equilibrium risk premiums to provide a neutral reference point for expected returns, generating a market-capitalisation-weighted-portfolio and then incorporating subjective views that tilts in the direction of assets favored by the investor to enable the generation of alpha. The Black-Litterman models assumes does not assume the model is always at CAPM equilibrium, but any shift away from this equilibrium will experience pressure from the market to revert. Furthermore, the model allows the investor to have as many, or as few, views as they wish on either an absolute or relative basis. 

The issue with the CAPM model and minimum-variance portfolio decribed by Markowitz is that small tweaks in the values could lead to significant rebalancing of the portfolio. The Markowitz framework effectively takes the risk-free return as the alternative benchmark. 


There functional x numbers of steps we need to follow:

- We need to asign weights to our portfolio based on a relevant benchmark or methodology. 
- We need to construct prior returns for each of our assets based on their relative weights
- We need to construct a matrix of investor views to cause our assets to diverge from the initial weights 
- We combine the market implied returns and investor views to get the posterior distribution
- We then generate our new allocations. 

We will use a 60/40 US portfolio as our benchmark we will use the reference ETFs SPY to proxy the equitiy portion of the portfolio and the fixed income portion will be represented via AGG. We will use 11 ETFs to represent the GICS sectors and we will weight these assets according to the market weights of the SPY reference index, and we will break down the fixed income allocation into short and long dated sectors with the appropriate weights according to AGG benchmark.

While the Markowitz min-variance optimisation is designed to use discrete returns, the Black-Litterman model is based on log return. 

$$

R_{i} = log(1 + \frac{P_{i+1}}{P_i})

$$

Whereby $ R_{d} = \frac{S_{i+1} - S_{i}}{S_i} $


Fig. 1 - Default Weights from S&P 500 per GICs Sector

| Sector                  | Weights  |
| :---------------------: | :---:    |
| Information Technology  | 30.37%   |
| Financials              | 12.95%   |
| Health Care             | 12.37%   |
| Consumer Discretionary  | 9.60%    |
| Communication           | 8.99%    |
| Industrials             | 8.44%    |
| Consumer Staples        | 6.16%    |
| Energy                  | 3.66%    |
| Utilities               | 2.51%    |
| Real Estate             | 2.39%    |
| Materials               | 2.28%    |
| Cash and/or Derivatives | 0.29%    |

#### 2.2 Reverse Optimisation & Prior Returns

Now that we have the optimal weights according to our equilibrium returns we are going to undertake a reverse optimisation of the weights to arrive at the returns vector for our sectors that would generate these weights. To solve this reverse optimisation we need to set up our problem. 

$$
arg \min_{\omega} \omega'\pi - \lambda\omega'\Sigma\omega
$$

- $\omega$: a vector of our sector weights
- $\pi$: a vector of our equilibrium returns
- $\lambda$: a scalar factor for risk-aversion
- $\Sigma$: covariance matrix for our returns

Initially we will solve an unconstrained mean-variance and then will introduce constraints for a more robust solution. We take the derivative with respect to $w$ to get the following equation: 

$$
\pi - 2 \lambda \Sigma w = 0
$$

We re-arrange the equation for both $\pi$ and $w$ to get the implied equilibrium return from our market weights and optimal weights under our model:

$$
\pi^{*} = 2 \lambda \Sigma w
$$

$$
w^{*} = \frac{1}{2 \lambda} \Sigma^{-1} \pi
$$

We can now calculate the implied market returns from our model based upon the market weights provided by the corresponding ETF proxies.

In [8]:
# table = tabulate((annualised_implied_excess_returns_vector.map('{:,.2f}%'.format)), headers='keys', tablefmt='pipe')

# print(table)

Fig. 2 - Implied Equilibrium Returns per GICs Sector (Annualised)

|                        | Implied Return   |
|:-----------------------|:-----------------|
| Energy                 | 7.45%            |
| Materials              | 7.28%            |
| Industrials            | 7.12%            |
| Consumer Discretionary | 6.70%            |
| Consumer Staples       | 4.88%            |
| Health Care            | 5.47%            |
| Financials             | 7.25%            |
| Information Technology | 8.06%            |
| Communication Services | 6.09%            |
| Utilities              | 4.18%            |
| Real Estate            | 6.06%            |

#### 2.3 Formulating Views

Naturally if we plug these implied returns back into our formula for the optimal weights we will arrive back at the market weights we initially used to derive the returns. The Black-Litterman model only suggests a deviation from the market returns once we have formulated views on assets. We need to specify our views on the market, which may or may not, clash with the reference market distribution. 

Views are expressed in either absolute or relative terms. We will define the following views:

- View 1: Energy will an absolute excess return 7%
- View 2: Information Technology will outperform Financials by 25bps on an excess return basis. 
- View 3: Subset defensive sectors (Consumer Staples & Utilities) will outperform the subset cyclical sectors (Consumer Discretionary & Information Technology) by 1%. 

Despite the fact that we have an absolute view and a relative view, this does not necessarily suggest that we should positively allocate to the asset that we expect to either return positively or outperform the other asset. We need to compare our prediction against the implied return from the existing market weights and determine whether our view is producing either a positive or negative deviation from the implied market return, and only in the event of a positive deviation would we then increase our potential allocation. In the case of our first view we have an implied excess return for Energy at 7.45%, and therefore an excess return of 2% would represent a decrease of 45bps. We should see a reduction in the market allocation to this sector. 

Similarly although we are predicting an outperform of Information Technology over Financials we need to evaluate what the existing out-or-underperformance between the two sectors. Once again we can see that the Information Technology sector currently outperforms Financials by 81bps. The prediction of 25bps would represent another negative bias towards Information Technolgoy and should see an increase in the allocation to Financials correspondingly. 

The calculation is more complicated when we evaluate multiple sectors against each other because of the interlinking allocation dynamics. We will need to form two mini portfolios with each sector given proportional weighting to it's market weights and then compare the performance of two portfolios to determine whether our relative prediction will generate a positive or negative bias towards the existing allocation. 

Fig. 3 Comparing Subset Sector Implied Excess Return
|                        | Market Weight   | Relative Subset Weight  | Weight Excess Return |
|:-----------------------|:---------------:|:-----------------------:|:-------------------:|
| Consumer Staples       | 6.16%           | 71.05%                  | 4.76%               |
| Utilities              | 2.51%           | 28.95%                  | 1.21%               |
|                        |                 | Total                   | 5.97%               |

|                        | Market Weight   | Relative Subset Weight  | Weight Excess Return |
|:-----------------------|:---------------:|:-----------------------:|:-------------------:|
| Information Technology | 30.37%          | 75.98%                  | 6.12%               |
| Consumer Discretionary | 9.60%           | 24.02%                  | 1.60%               |
|                        |                 | Total                   | 7.72%               |

As we can see the current subset defensive sectors are significantly under performing the cycling sectors, and if we are confident in our view we should see a noticeable deviation in our new portfolio allocations compared to the existing market portfolio. 

Now that we have formulated our views on the market and what we expect to see from our allocations. Let us move onto the practical implementation of the views into our model mathemtically 

#### 2.4 Views Matrix

$Q$ corresponds to our views vector. We have 3 views so we will generate a $k \times 1$ vector where k is the number of views. Each view contains a degree of uncertainty about our confidence in the view and this is represented by $ \epsilon $. This term $\epsilon$ is a random, unknown, independent, normally distributed error term with a distribution as follows:

$$
\epsilon \sim \mathcal{N}(0, \Omega)
$$

The error term itself does not enter our Black Litterman model but the difference between the error terma and the expected value of 0 will enter the model. The variance represents the uncertainty, and therefore the larger the variance the greater the uncertainty. 

The expressed views in our $Q$ vector is then mapped to the corresponding asset by the matrix $P$. Each expressed view generates a $1 \times N$ row vector and ultimately gives us a $K \times N$ matrix. The difference between an absolute view and a relative view in our matrix is defined by the row total for each option. Relative views will sum to 0 on each row, while Absolute views will not. In the case of our first row we have an absolute view on the 1st asset. As a result the row vector will be defined as below:

$$
P_1 = \begin{bmatrix}
    1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0
    \end{bmatrix}
$$

The second view is a relative view on two assets so the nominally outpeforming asset will recieve a positive weighting while the underperforming asset will recieve a negative rating, however the row must sum to 0. We will apply these weights to the 7th and 8th assets. 

$$
P_2 \begin{bmatrix}
    0 & 0 & 0 & 0 & 0 & 0 & -1 & 1 & 0 & 0 & 0 
    \end{bmatrix}
$$

In the event that we are comparing muliple assets for a relative view then the situation is slightly more complicated. The row still needs to sum up to 0 but we need away to assign values to 4 assets rather than just 2. Satchell and Scowcroft demonstrate the use of an equal weighting scheme, whiile Idzorek uses a market capitalisation approach. Following Satchell and Scowcroft approach leads to the simplest vector by simply applying a positive 2/N and a negative 2/N value to outperforming and underperforming assets respectively. 

$$
P_3 = \begin{bmatrix}
    0 & 0 & 0 & -0.5 & 0.5 & 0 & 0 & -0.5 & 0 & 0.5 & 0
    \end{bmatrix}
$$

This is the simplest formulation for our problem but because it ignores the portfolio weights and therefore can lead to more extreme rebalancing on smaller weighted sectors. Idzorek proposes that we introduce the relative subset weight for our sectors instead to mitigate this overreaction in re-allocation and given we already calculate them above it's fairly simple to substitute them in and get our final views matrix. 

$$
P = \begin{bmatrix}
    1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
    0 & 0 & 0 & 0 & 0 & 0 & -1 & 1 & 0 & 0 & 0 \\
    0 & 0 & 0 & -0.24 & 0.71 & 0 & 0 & -0.76 & 0 & 0.29 & 0
    \end{bmatrix}
$$


In [9]:
# TODO: scalar and covariance matrix of the error term

Views are expressed on the expected returns $P_u$ and have the following distribution:

$$
P_{u} \sim \mathcal{N}(v, \Omega)
$$

and our covariance matrix of error terms is $ \Omega = diag(P(\tau \Sigma) P^{'}) $

The term $\tau$ is a scalar that controls the value assigned to the uncertainty to the combined returns distribution

In [10]:
# define the implied equity adjusted returns using Bayes Thereom via Meucci (2010)

$$
\mu_{BL} = \pi + \tau \Sigma P^{'} (\tau P \Sigma P^{'} + \Omega)^{-1}(v - P\pi)
$$

Now that we have the posterior returns for our sectors we can now obtain the optimal weights for this new distribution. Our new optimisation problem follows:

$$
arg \min_{\omega} \omega'\mu_{BL} - \lambda\omega'\Sigma\omega
$$


As we saw earlier in an unconstrained problem this gives us the neat solution:

$$
w^{*} = \frac{1}{2 \lambda} \Sigma^{-1} \mu_{BL}
$$

We can now compare across all our two return distributions and subsequent changes in market allocation. 


Fig 4. Comparing the change in sector return and weights before and after the introduction of views

|                        | Adjusted Return   | Implied Return   | Adj. - Impl. Return   | Adjusted Weights   | Equilibrium Weights   | Adj. - Eq. Weights   |
|:-----------------------|:------------------|:-----------------|:----------------------|:-------------------|:----------------------|:---------------------|
| Energy                 | 6.77%             | 7.45%            | -0.67%                | 4.91%              | 3.66%                 | 1.25%                |
| Materials              | 6.39%             | 7.28%            | -0.89%                | 2.26%              | 2.28%                 | -0.02%               |
| Industrials            | 6.16%             | 7.12%            | -0.97%                | 8.36%              | 8.44%                 | -0.08%               |
| Consumer Discretionary | 5.46%             | 6.70%            | -1.24%                | -0.25%             | 9.60%                 | -9.85%               |
| Consumer Staples       | 4.85%             | 4.88%            | -0.04%                | 34.96%             | 6.16%                 | 28.80%               |
| Health Care            | 4.88%             | 5.47%            | -0.58%                | 12.25%             | 12.37%                | -0.12%               |
| Financials             | 6.33%             | 7.25%            | -0.92%                | 5.08%              | 12.95%                | -7.87%               |
| Information Technology | 6.11%             | 8.06%            | -1.95%                | 6.91%              | 30.37%                | -23.46%              |
| Communication Services | 5.36%             | 6.09%            | -0.73%                | 8.90%              | 8.99%                 | -0.09%               |
| Utilities              | 4.60%             | 4.18%            | 0.42%                 | 14.27%             | 2.51%                 | 11.76%               |
| Real Estate            | 5.69%             | 6.06%            | -0.36%                | 2.37%              | 2.39%                 | -0.02%               |

The adjusted return after incorporating out views are consistently, with the exception of utilities, lower than the implied marke returns from our model. This is intuitively inline with our views, we expected energy to perform worse on an absolute basis, the gap between Financials and Information technology has significantly shrink and we expected for our defensive subsectors to perform better than our growth sectors. Curiously this has manifesting in a significant reduction in predicted returns for our Information Technology and Consumer Discretionary, while Consumer Staples and Utilities maintain and slightly overperform respectively. Naturally, this new adjusted return vector does lead to significant reblancing in our sectors specified in Views 3 and 2, but we also seem minimal tweaks of the other sectors in the portfolio. This is confirmation of the in-built theory within the Black-Litterman model that deviating from the market portfolio is only appropraite when there is a mismatch between market expectations and the current equilibrium. 

In [11]:
# table = tabulate(pre_post_optimisation.applymap(lambda x: f"{x:.2%}"), headers='keys', tablefmt='pipe')

# print(table)

#### 2.5 Adding constraints to our optimisation

Now that we have our default Black-Litterman model defined and initailised we can move onto producing a more robust solution to our portfolio construction. Currently we are performing unconstrained opitmision but in reality investors are commonly constrained. Currently in our unconstrained portfolio we are taking a small short position in Consumer Discretionary which we want to eliminate for a long only portfolio. We can add two fairly common investor constraints for no leverage and no short positions. 

Our maximisation problem is formulated slightly differently now:

$$
arg \max_{\omega} \omega'\pi - \lambda\omega'\Sigma\omega
$$

s.t.

$$
w_{i} \geq 0
$$

$$
\sum_{i=1}^{n} w_{i} = 1

$$

- $\omega$: a vector of our sector weights
- $\pi$: a vector of our equilibrium returns
- $\lambda$: a scalar factor for risk-aversion
- $\Sigma$: covariance matrix for our returns
- $\textbf{1}$: This is a vector of 1

When we express a constraint to quadprog() we need to express it in the following form:

$$
\textbf{C}' \textbf{w} \geq \textbf{b}
$$

- $\textbf{C}$: is the matrix that represents the coefficients of the weights $w_{i}$
- $\textbf{b}$: is a matrix representing the right hand side of our inequality. 

We need to re-formulate our two constraints into similar formats so that we know how to define our two matrix inputs into our model. We can begin with our no-short selling constraint and convert into matrix form: 

For a portfolio of $n$ assets, the no-short selling constraint can be written as the following set of inequalities: 

$$
w_1 \geq 0, \quad w_2 \geq 0, \quad \dots, \quad w_n \geq 0
$$

We can therefore represent this in matrix form as: 

$$
\begin{bmatrix}
1 & 0 & \dots & 0 \\
0 & 1 & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & 1
\end{bmatrix}
\begin{bmatrix}
w_1 \\
w_2 \\
\vdots \\
w_n
\end{bmatrix}
\geq
\begin{bmatrix}
0 \\
0 \\
\vdots \\
0
\end{bmatrix}
$$

This means that if we want to represent the contrainst in the same format as the quadprog() is expecting we get the following values:

- $\textbf{C} = \textbf{I}$, where $I$ is an identity matrix of size $n \times n$
- $\textbf{b} = \textbf{0}$, a vector of length $n$

For our no leverage constraint it is straightforward to convert our summation notion into a matrix form:

$$
\begin{bmatrix}
1 & 1 & \dots & 1
\end{bmatrix}
\begin{bmatrix}
w_1 \\
w_2 \\
\vdots \\
w_n
\end{bmatrix}
=
1
$$

- $\textbf{C} = \textbf{1}$, where $1$ is an $1 \times n$ matrix and every value is 1.
- $\textbf{b} = 1$, an integer value

Now that we have successfully reformulated our constraints in a format amenable to quadprog() we have our constraint matrices:

$$
C = 
\begin{bmatrix}
\textbf{I} \\
\textbf{1}
\end{bmatrix}
$$

$$
b = 
\begin{bmatrix}
\textbf{0} \\
1
\end{bmatrix}
$$

In [12]:
# table = tabulate(weghts_comparision.applymap(lambda x: f"{x:.2%}"), headers='keys', tablefmt='pipe')

# print(table)

Fig 5. Comparing the equilibrium weights to the constrained and unconstrained weights

|                        | Equilibrium Weights   | Adjusted Weights   | Constrained Adjusted Weights   |
|:-----------------------|:----------------------|:-------------------|:-------------------------------|
| Energy                 | 3.66%                 | 4.93%              | 4.98%                          |
| Materials              | 2.28%                 | 2.26%              | 2.33%                          |
| Industrials            | 8.44%                 | 8.35%              | 8.48%                          |
| Consumer Discretionary | 9.60%                 | -0.25%             | 0.00%                          |
| Consumer Staples       | 6.16%                 | 34.93%             | 34.62%                         |
| Health Care            | 12.37%                | 12.24%             | 11.99%                         |
| Financials             | 12.95%                | 5.10%              | 5.26%                          |
| Information Technology | 30.37%                | 6.91%              | 6.93%                          |
| Communication Services | 8.99%                 | 8.90%              | 8.78%                          |
| Utilities              | 2.51%                 | 14.26%             | 14.17%                         |
| Real Estate            | 2.39%                 | 2.37%              | 2.45%                          |

Given we had already normalised the weights on our unconstrained opitimisation there is minimal change with the introduction of formal constraints, however we can see that our Consumer Discretionary allocation has adjusted to 0 rather than a small short. This is confirmation that our constraints have been incorporated correctly. 

TODO: Compare across the different risk profiles?

2.7 Tracking Error

Tracking error is defined as the difference between the portfolio return and the benchmark return over a defined period and a given frequency. 

$$
TE = \sqrt{\frac{1}{T - 1} \sum_{t=1}^{T} (R_p(t) - R_b(t))^2}
$$

$R_p(t)$: is the portfolio return at time $t$
$R_b(t)$: is the benchmark return at time $t$
$T$: is the number of time periods

Most active managers are benchmarked against a specific index, in our framework we have replicated the S&P 500 via individual sectors and are seeking to incorporate views into our market portfolio to generate alpha. However, by deviating from the market we are taking on active risk and currently we have no way to determine the strength of our views. We can implement a Tracking Error constraint following the methodology discussed 

We can add an additional constraint to our constrained mean-variance 



P Jorion
We define the following notations:

- $q$ = vector of index weights for the sample of $N$ assets
- $x$ = vector of deviations from index
- $q_p = q + x$ = vector of portfolio weights
- $E$ = vector of expected returns
- $V$ = covariance matrix for asset returns

We can write expected returns and variances in matrix notation:

- $\mu_b = q'E $ = expected return on index
- $\sigma_b^2 = q'Vq $ = variance of index return
- $\mu_e = x'E $ = expected excess return
- $\sigma_e^2 = T = x'Vx $ = variance of tracking error

The active portfolio expected return and variance:

- $\mu_p = (q + x)' E $
- $\sigma_p^2 = (q + x)'V(q + x) $

We can now set up our optimisation problem:

$$
max \space x'E
$$

$$
s.t.
$$

$$
x'1 = 0
$$

$$
x'Vx = T
$$

The first 

We can adapt this formulation defined by Jorion to our Black-Litterman model by substituting our market implied returns $\pi$ for $E$. Once we have the optimal deviations $x*$ then we simply need to combine our values with our existing market weights to get your optimal portfolio weights in a Black-Litterman framework. 








2.6 Optimising for Sharpe Ratio

2.7 Confidence Intervals

# ------------------- Python -----------------------------

#### 1 Data & Pre-processing

In [22]:
df = pd.read_sql('portfolio_data', con=engine)

# # set date as index
df.set_index('Date', inplace=True)

In [18]:
df = pd.read_sql('portfolio_data', con=engine)

# # set date as index
df.set_index('Date', inplace=True)

# create separate 
three_month_tbill = df['3M TB'] / 252

# consistent ordering
df = df[['Energy', 'Materials', 'Industrials', 'Consumer Discretionary', 'Consumer Staples', 'Health Care', 'Financials', 'Information Technology', 
         'Communication Services', 'Utilities', 'Real Estate', 'Volatility (Exo)', 'Commodities (Exo)', 'Bonds (Exo)']]
        
# calculate simple returns
simple_returns = df.pct_change()

# subtract the 3m t bill daily rate
excess_returns = simple_returns.sub(three_month_tbill, axis=0)

# # calculate log returns returns
log_returns = np.log(1 + simple_returns).dropna()

/var/folders/3z/lz8knlfn5ys693qpn6bnjb440000gn/T/ipykernel_33994/3768494725.py:31: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  simple_returns = df.pct_change()


In [19]:
# Table of Assets

pd.DataFrame

df.columns

Index(['Energy', 'Materials', 'Industrials', 'Consumer Discretionary',
       'Consumer Staples', 'Health Care', 'Financials',
       'Information Technology', 'Communication Services', 'Utilities',
       'Real Estate', 'Volatility (Exo)', 'Commodities (Exo)', 'Bonds (Exo)'],
      dtype='object')

#### Black-Litterman Class 

In [345]:
class BlackLittermanModel:
    def __init__(self, equilibrium_weights, log_returns, risk_aversion, views_mapping_matrix, views_matrix, tracking_error_target=0.00000009, tau=0.025):
        self._equilibrium_weights = equilibrium_weights
        self._log_returns = log_returns
        self._risk_aversion = risk_aversion
        self._tau = tau
        self._covariance_matrix = self._calculate_covariance_matrix()
        self._views_mapping_matrix = views_mapping_matrix
        self._views_matrix = views_matrix
        self._tracking_error_target = tracking_error_target 

    # TODO: private methods
    def _calculate_covariance_matrix(self):
        # calculate the vols
        volatilities_array = self._log_returns.std()

        # calculate correlation
        correlation_coefficients = self._log_returns.corr()

        # create the diagonal vol matrix (vol on the diagonal, zeros elsewhere)
        std_diag_vol_matrix  = np.diag(volatilities_array)

        # compute the covariance matrix
        covariance_matrix = std_diag_vol_matrix @ correlation_coefficients.values @ std_diag_vol_matrix

        # pass the headers from log return 

        covariance_matrix = pd.DataFrame(
            covariance_matrix,
            columns = self._log_returns.columns,
            index = self._log_returns.columns,
        )

        return covariance_matrix
    
    def calculate_views_adjusted_returns(self):
        # we need implied equity returns and omega
        implied_returns_vector = self.calculate_implied_equilibrium_returns().values

        omega = self.calculate_uncertainty_views_matrix()

        # we have three terms for the np.dot product
        tau_sigma_transpose_p = self._tau * (self._covariance_matrix.values @ self._views_mapping_matrix.T)

        inverse_middle_term = np.linalg.inv((self._views_mapping_matrix @ tau_sigma_transpose_p) + omega)

        view_minus_p = (self._views_matrix - (self._views_mapping_matrix @ implied_returns_vector))

        views_adjusted_returns_vector = implied_returns_vector + ((tau_sigma_transpose_p @ inverse_middle_term) @ view_minus_p)

        # convert to dataframe for consistency
        views_adjusted_returns = pd.DataFrame(views_adjusted_returns_vector, columns=["Adjusted Return"], index=self._log_returns.columns)

        return views_adjusted_returns
    
    # calculate the covariance of the error terms aka omega
    def calculate_uncertainty_views_matrix(self):
        # you extract the diagonal and then convert to a diagonal matrix with zeros
        omega = np.diag(((self._views_mapping_matrix @ (self._tau * self._covariance_matrix)) @ self._views_mapping_matrix.T))
        
        return np.diag(omega)

    
    def calculate_implied_equilibrium_returns(self):
        implied_returns_vector = self._risk_aversion * (self._covariance_matrix @ self._equilibrium_weights)

        implied_returns = pd.DataFrame(implied_returns_vector, columns=["Implied Return"], index=self._log_returns.columns)

        return implied_returns
    
    def calculate_unconstrained_mv_optimisation(self):
        # invert the covariance matrix
        inv_cov_matrix = np.linalg.inv(self._covariance_matrix)

        # calculate adjusted returns
        adjusted_returns = self.calculate_views_adjusted_returns().values

        # obtain weights
        optimal_weights = self._risk_aversion * np.dot(inv_cov_matrix, adjusted_returns)

        # # normalise the sum of weights to 1
        optimal_weights /= np.sum(optimal_weights)

        # convert to dataframe for consistency
        optimal_weights = pd.DataFrame(optimal_weights, columns=["Adjusted Weights"], index=self._log_returns.columns)

        return optimal_weights
    
    def generate_mv_constraint_matrices(self, implied_returns_vector_values):
        # we will generate our two constraints - no short selling & no leverage
        number_of_assets = len(implied_returns_vector_values)

        # no short selling
        no_short_selling_constraint_coefficient = np.eye(number_of_assets)
        no_short_selling_constraint_rhs = np.zeros(number_of_assets)

        # no leverage - you need to transpose the 
        no_leverage_constraint_coefficient = np.ones([1, number_of_assets])
        no_leverage_constraint_rhs = np.array([1.0])

        # stack them together
        C_constraint_matrix = np.vstack([no_leverage_constraint_coefficient, no_short_selling_constraint_coefficient])
        b_constraint_matrix = np.hstack([no_leverage_constraint_rhs, no_short_selling_constraint_rhs])

        return C_constraint_matrix, b_constraint_matrix
    
        
    def calculate_constrained_mv_optimisation(self):
        implied_returns_vector_values = self.calculate_implied_equilibrium_returns().values

        # we are solving an equation of the form - 1/2 x^T G x - a^T x s.t. C.T x >= b
        quadratic_G = self.risk_aversion * self._covariance_matrix.values

        quadratic_a = self.calculate_views_adjusted_returns().values.flatten()

        # generate constraints
        C_constraint_matrix, b_constraint_matrix = self.generate_mv_constraint_matrices(implied_returns_vector_values)

        # solve for weights
        optimal_weights = quadprog.solve_qp(quadratic_G, quadratic_a, C_constraint_matrix.T, b_constraint_matrix, meq=1)[0]

        optimal_weights = pd.DataFrame(optimal_weights, columns=["Constrained Adjusted Weights"], index=self._log_returns.columns)

        return optimal_weights
    
    
    def calculate_tracking_error_optimisation(self):
        implied_returns_vector_values = self.calculate_implied_equilibrium_returns().values

        number_of_assets = len(implied_returns_vector_values)

        adjusted_returns = self.calculate_views_adjusted_returns().values

        # constraints
        constraints = [{'type': 'eq', 'fun': self.zero_sum_equality_constraint},
                       {'type': 'eq', 'fun': self.tracking_error_target_constraint}]
        

        # bounds - no short selling or other constraints
        bounds = [(None, None)] * number_of_assets

        # initial guess - very small non-zero deviation
        x0 = np.ones(number_of_assets) * 1e-3

        # generate weights
        results = minimize(fun= self.objective_function, x0=x0, bounds=bounds, constraints=constraints)
        
        if not results.success:
            print("Optimization failed:", results.message)
            return None

        optimal_deviations = results.x

        # create a series from the deviations with the same index as the eq weights
        optimal_deviations = pd.Series(results.x, index=self._equilibrium_weights.index)

        optimal_weights = self._equilibrium_weights + optimal_deviations

        optimal_weights = pd.DataFrame(optimal_weights, columns=["TE Constrained Adjusted Weights"], index=self._log_returns.columns)

        return optimal_weights
    
    # Tracking Error - objective and constraints
    def objective_function(self, x):
        # maximise returns
        adjusted_returns = self.calculate_views_adjusted_returns().values
        value = - np.dot(x, adjusted_returns)
        # print(f"Objective function value: {value} for x: {x}")


        return value
    
    def zero_sum_equality_constraint(self, x):
        # constraint - sum of deviations is zero
        value = np.sum(x)
        # print(f"Zero sum constraint value: {value} for x: {x}")

        return value
    
    def tracking_error_target_constraint(self, x):
        value = np.dot(x.T, np.dot(self.covariance_matrix, x)) - self._tracking_error_target
        # print(f"Tracking error constraint value: {value} for x: {x}")

        return value


    # attributes
    # tau
    @property
    def tau(self):
        return self._tau

    @tau.setter
    def tau(self, value):
        self._tau = value

    # risk aversion 
    @property
    def risk_aversion(self):
        return self._risk_aversion

    @risk_aversion.setter
    def risk_aversion(self, value):
        self._risk_aversion = value

    @property
    def covariance_matrix(self):
        return self._covariance_matrix

In [330]:
# testing the class
equilibrium_weights = pd.Series({
    'Energy': 0.0366,
    'Materials': 0.0228,
    'Industrials': 0.0844,
    'Consumer Discretionary': 0.0960,
    'Consumer Staples': 0.0616,
    'Health Care': 0.1237,
    'Financials': 0.1295,
    'Information Technology': 0.3037,
    'Communication Services': 0.0899,
    'Utilities': 0.0251,
    'Real Estate': 0.0239
})

risk_aversion_dict = pd.Series({
    # in order of increasing aversion
    'Kelly': 0.01,
    'Market': 2.24,
    'Trustee': 6
})

# view adjusted returns with defined views
k = 3
n = 11

#  these are annual views so we need to divide by 252 to make them daily
Q = np.array([0.07, 0.0025, 0.01]).reshape(-1, 1) / 252

P = np.zeros((k, n))

# bl_model.calculate_views_adjusted_returns()
# first view
P[0, 0] = 1
# second view
P[1, 6] = -1
P[1, 7] = 1
# third view
P[2, 3] = -0.24
P[2, 4] = 0.71
P[2, 7] = -0.76
P[2, 9] = 0.29

bl_model = BlackLittermanModel(equilibrium_weights, log_returns, risk_aversion_dict['Market'], P, Q, tracking_error_target=0.1)

# bl_model.calculate_implied_equilibrium_returns()

# bl_model.calculate_views_adjusted_returns().mul(100).mul(252)

In [331]:
bl_model.calculate_unconstrained_mv_optimisation().mul(100).round(4)

# bl_model.calculate_views_adjusted_returns().mul(100).mul(252)

# bl_model.calculate_unconstrained_mv_optimisation().mul(100)

# bl_model.calculate_views_adjusted_returns()

,Adjusted Weights
Energy,4.9310
Materials,2.2565
Industrials,8.3529
Consumer Discretionary,-0.2484
Consumer Staples,34.9383
Health Care,12.2424
Financials,5.1042
Information Technology,6.8959
Communication Services,8.8973
Utilities,14.2646


In [332]:
# bl_model.calculate_implied_equilibrium_returns().mul(100).mul(252)['Optimal We'] - bl_model.calculate_views_adjusted_returns().mul(100).mul(252)

In [334]:
bl_model = BlackLittermanModel(equilibrium_weights, log_returns, risk_aversion_dict['Market'], P, Q, tracking_error_target=0.0000001)

bl_model.calculate_tracking_error_optimisation().mul(100)

,TE Constrained Adjusted Weights
Energy,5.270487
Materials,3.033214
Industrials,8.767928
Consumer Discretionary,9.545518
Consumer Staples,5.379024
Health Care,11.709656
Financials,13.393171
Information Technology,30.603811
Communication Services,8.703778
Utilities,1.267696


In [322]:
bl_model._equilibrium_weights

Energy                    0.0366
Materials                 0.0228
Industrials               0.0844
Consumer Discretionary    0.0960
Consumer Staples          0.0616
Health Care               0.1237
Financials                0.1295
Information Technology    0.3037
Communication Services    0.0899
Utilities                 0.0251
Real Estate               0.0239
dtype: float64

In [323]:
# returns
annualised_adjusted_excess_returns_vector = bl_model.calculate_views_adjusted_returns().mul(252)
annualised_implied_excess_returns_vector = bl_model.calculate_implied_equilibrium_returns().mul(252)

# weights
adjusted_weights_vector = bl_model.calculate_unconstrained_mv_optimisation()
equilibrium_weights_vector = pd.DataFrame(equilibrium_weights, columns=['Equilibrium Weights'])

pre_post_optimisation = pd.concat([annualised_adjusted_excess_returns_vector, 
                         annualised_implied_excess_returns_vector,
                         adjusted_weights_vector,
                         equilibrium_weights_vector], axis=1)


# delta weights and return
pre_post_optimisation['Adj. - Impl. Return'] = pre_post_optimisation['Adjusted Return'] - pre_post_optimisation['Implied Return']
pre_post_optimisation['Adj. - Eq. Weights'] = pre_post_optimisation['Adjusted Weights'] - pre_post_optimisation['Equilibrium Weights']

# column order
pre_post_optimisation = pre_post_optimisation[['Adjusted Return', 'Implied Return', 'Adj. - Impl. Return', 'Adjusted Weights', 'Equilibrium Weights', 'Adj. - Eq. Weights']]

pre_post_optimisation.applymap(lambda x: f"{x:.2%}")

/var/folders/3z/lz8knlfn5ys693qpn6bnjb440000gn/T/ipykernel_20667/1318777385.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pre_post_optimisation.applymap(lambda x: f"{x:.2%}")


,Adjusted Return,Implied Return,Adj. - Impl. Return,Adjusted Weights,Equilibrium Weights,Adj. - Eq. Weights
Energy,6.77%,7.44%,-0.67%,4.93%,3.66%,1.27%
Materials,6.39%,7.27%,-0.88%,2.26%,2.28%,-0.02%
Industrials,6.15%,7.12%,-0.97%,8.35%,8.44%,-0.09%
Consumer Discretionary,5.46%,6.70%,-1.24%,-0.25%,9.60%,-9.85%
Consumer Staples,4.84%,4.88%,-0.03%,34.94%,6.16%,28.78%
Health Care,4.88%,5.47%,-0.58%,12.24%,12.37%,-0.13%
Financials,6.33%,7.25%,-0.92%,5.10%,12.95%,-7.85%
Information Technology,6.11%,8.06%,-1.95%,6.90%,30.37%,-23.47%
Communication Services,5.36%,6.09%,-0.73%,8.90%,8.99%,-0.09%
Utilities,4.59%,4.17%,0.42%,14.26%,2.51%,11.75%


In [324]:
# weights
equilibrium_weights_vector = pd.DataFrame(equilibrium_weights, columns=['Equilibrium Weights'])
unconstrained_weights_vector = bl_model.calculate_unconstrained_mv_optimisation()
constrained_weights_vector = bl_model.calculate_constrained_mv_optimisation()

weghts_comparision = pd.concat([equilibrium_weights_vector,
                         unconstrained_weights_vector,
                         constrained_weights_vector], axis=1)


# delta weights and return
# combined_df['Adj. - Impl. Return'] = combined_df['Adjusted Return'] - combined_df['Implied Return']
# combined_df['Adj. - Eq. Weights'] = combined_df['Adjusted Weights'] - combined_df['Equilibrium Weights']

# column order
# combined_df = combined_df[['Adjusted Return', 'Implied Return', 'Adj. - Impl. Return', 'Adjusted Weights', 'Equilibrium Weights', 'Adj. - Eq. Weights']]

weghts_comparision.applymap(lambda x: f"{x:.2%}")

/var/folders/3z/lz8knlfn5ys693qpn6bnjb440000gn/T/ipykernel_20667/51984312.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weghts_comparision.applymap(lambda x: f"{x:.2%}")


,Equilibrium Weights,Adjusted Weights,Constrained Adjusted Weights
Energy,3.66%,4.93%,4.98%
Materials,2.28%,2.26%,2.33%
Industrials,8.44%,8.35%,8.48%
Consumer Discretionary,9.60%,-0.25%,0.00%
Consumer Staples,6.16%,34.94%,34.62%
Health Care,12.37%,12.24%,11.99%
Financials,12.95%,5.10%,5.27%
Information Technology,30.37%,6.90%,6.92%
Communication Services,8.99%,8.90%,8.78%
Utilities,2.51%,14.26%,14.17%


In [351]:
bl_model = BlackLittermanModel(equilibrium_weights, log_returns, risk_aversion_dict['Market'], P, Q, tracking_error_target=0.000001)

adjusted_retun_vector = bl_model.calculate_views_adjusted_returns().mul(252)
equilibrium_weights_vector = pd.DataFrame(equilibrium_weights, columns=['Equilibrium Weights'])
tracking_error_weights = bl_model.calculate_tracking_error_optimisation()

weghts_comparision = pd.concat([adjusted_retun_vector,
                                equilibrium_weights_vector, 
                                tracking_error_weights], axis=1)

weghts_comparision.applymap(lambda x: f"{x:.2%}")

/var/folders/3z/lz8knlfn5ys693qpn6bnjb440000gn/T/ipykernel_20667/493338855.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weghts_comparision.applymap(lambda x: f"{x:.2%}")


,Adjusted Return,Equilibrium Weights,TE Constrained Adjusted Weights
Energy,6.77%,3.66%,8.05%
Materials,6.39%,2.28%,4.80%
Industrials,6.15%,8.44%,9.80%
Consumer Discretionary,5.46%,9.60%,9.39%
Consumer Staples,4.84%,6.16%,2.95%
Health Care,4.88%,12.37%,9.80%
Financials,6.33%,12.95%,14.86%
Information Technology,6.11%,30.37%,31.66%
Communication Services,5.36%,8.99%,8.03%
Utilities,4.59%,2.51%,-1.64%


In [346]:
bl_model.calculate_views_adjusted_returns().mul(252)

,Adjusted Return
Energy,0.067684
Materials,0.063862
Industrials,0.061534
Consumer Discretionary,0.054635
Consumer Staples,0.048448
Health Care,0.048815
Financials,0.063324
Information Technology,0.061104
Communication Services,0.053611
Utilities,0.045928


Bibliography

Black, F. and Litterman, R. 1990. Asset Allocation: Combining Investors Views with Market Equilibrium. Goldman Sachs Fixed Income Research working paper

Black, F. and Litterman, R. 1991. Global Asset Allocation with Equities, Bonds, and Currencies. Goldman Sachs Fixed Income Research working paper

Black, F. and Litterman, R. 1992. Global Portfolio Optimization.Financial Analysts Journal, 28-43.

Idzorek, T.M. 2002. A step-by-step guide to Black-Litterman model. Incorporating user-specified confidence levels. Working Paper, 2-11.

Satchell, S. and Scowcroft, A. (2000). “A Demystification of the Black-Litterman Model: Managing Quantitative and Traditional Construction.” Journal of Asset Management, September, 138-150.

Markowitz, H., 1952. Portfolio Selection. The Journal of Finance, 7(1): 77-91.

Fama and French Data Library - http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

Fama, Eugene F. and French, Kenneth R., Production of U.S. Rm-Rf, SMB, and HML in the Fama-French Data Library (December 18, 2023). Chicago Booth Research Paper No. 23-22, Fama-Miller Working Paper

